# Imports

In [36]:
import torch
import torch.nn as nn
import matplotlib
import matplotlib.pyplot as plt
import os
import numpy as np
%matplotlib inline

# CNN architecture

In [37]:
class Cifar10CnnModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            
            # Conv-1
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(0.25),
            
            # Conv-2
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AvgPool2d(2, 2), # output: 64 x 16 x 16
            
            # Conv-3
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(0.25),

            # Conv-4
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.AvgPool2d(2, 2), # output: 128 x 8 x 8

            # Conv-5
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Dropout(0.25),            
            
            # Conv-6
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),           
            nn.ReLU(),
            nn.AvgPool2d(2, 2), # output: 256 x 4 x 4

            nn.Flatten(), 
            nn.Linear(256*4*4, 32),
            nn.Dropout(0.25),            
            nn.ReLU(),
            nn.Linear(32, 10))
        
    def forward(self, xb):
        return self.network(xb)

# Loading saved model

In [38]:
saved_model = Cifar10CnnModel()
saved_model.load_state_dict(torch.load('../saved_model/cifar10-cnn_with_bn_and_avg_pool.pth', map_location='cpu'))
saved_model.eval()

Cifar10CnnModel(
  (network): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.25, inplace=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.25, inplace=False)
    (12): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (16): Conv2d(128, 256, kernel_size

# Helper functions

In [39]:
def create_if_not_exists(base_path, name_of_folder=None):
    path = base_path + "/" + name_of_folder + "/"
    try:
        if name_of_folder != None:
            os.makedirs(base_path + "/{}".format(name_of_folder))
        else:
            os.makedirs(base_path)
    except FileExistsError:
        pass
    
    return path

def write_tensor_to_file(tensor, file_path, name_of_file):
    text_file = open(file_path + "/{}".format(name_of_file), "w+")
    tensor.data.numpy().tofile(text_file, sep=" ", format="%.8f")
    text_file.close()

# Convolutional layers

### Collecting all convolutional layers parameters (weight and bias)

#### Storing each kernel weight

In [40]:
conv_weight_list = []

for i in range(0, 24, 4):
    conv_weight_list.append("################# LAYER #################")
    conv_weight_list.append(saved_model.network[i].weight)

#### Storing each kernel bias

In [41]:
conv_bias_list = []

for i in range(0, 24, 4):
    conv_bias_list.append("################# LAYER #################")
    conv_bias_list.append(saved_model.network[i].bias)

### Creating a directory to store weights and biases parameters

In [42]:
path_where_all_parameters_be_stored = "./PrunedResnetData/all_parameters"

create_if_not_exists("./PrunedResnetData", "all_parameters")

'./PrunedResnetData/all_parameters/'

## Creating a directory to store weights

In [43]:
weight_path = create_if_not_exists(path_where_all_parameters_be_stored, "weights")

In [44]:
weight_path

'./PrunedResnetData/all_parameters/weights/'

### Saving convolutional layer's kernel weights

In [45]:
layer = 1
for i in range(0,len(conv_weight_list)):
    if type(conv_weight_list[i]) != str:
        conv_weight_tensor = conv_weight_list[i]
        write_tensor_to_file(conv_weight_tensor, weight_path, "P0-conv_layer_{}_weight.txt".format(layer))
        layer += 1

## Creating a directory to store biases

In [46]:
bias_path = create_if_not_exists(path_where_all_parameters_be_stored, "bias")

In [47]:
bias_path

'./PrunedResnetData/all_parameters/bias/'

### Saving convolutional layer's biases

In [48]:
layer = 1
for i in range(0,len(conv_bias_list)):
    if type(conv_bias_list[i]) != str:
        conv_bias_tensor = conv_bias_list[i]
        write_tensor_to_file(conv_bias_tensor, bias_path, "P0-conv_layer_{}_bias.txt".format(layer))
        layer += 1

# Batch normalization layers

## Getting mean

In [49]:
mean_list = []

layer = 1
for i in range (1, 7):
    print("Layer: ", layer)
    mean = saved_model.network[layer].running_mean
    print("Mean: \n", mean)
    mean_list.append(mean.tolist())
    layer += 4

Layer:  1
Mean: 
 tensor([-0.0572,  0.0268, -0.0598, -0.0056, -0.1814,  0.0054,  0.0091,  0.1488,
        -0.0084,  0.1128,  0.0717,  0.0393, -0.1927, -0.0054, -0.2938,  0.0249,
         0.0321, -0.0687, -0.0831, -0.0570,  0.1712,  0.0577,  0.0970, -0.0497,
         0.0561,  0.2250, -0.1915,  0.0176,  0.0793, -0.1571, -0.2537,  0.0589])
Layer:  5
Mean: 
 tensor([-5.6772e-01, -1.1758e+00, -8.6809e-01, -9.2560e-01,  6.3968e-01,
        -6.5766e-01, -1.5113e+00, -1.5153e+00, -1.2579e+00, -4.8949e-01,
        -3.7667e-01, -5.5906e-01, -3.7419e-01, -6.9433e-01, -1.6518e+00,
        -5.8868e-01, -1.3621e+00, -1.0631e+00, -2.4567e-04, -5.8325e-01,
        -4.8372e-01, -3.0479e+00, -2.2275e-01, -1.0783e+00, -1.2605e+00,
        -1.4628e+00, -1.2812e+00, -1.3137e-01, -7.0710e-01, -4.4702e-01,
        -1.6044e+00, -1.0181e+00,  9.0859e-01, -7.7269e-01, -1.6930e-01,
         1.9853e-01, -1.3136e+00, -2.9400e+00,  2.0589e-01, -6.6592e-01,
        -5.4606e-01, -4.7869e-01, -6.4273e-01, -1.6618e+00,

## Getting variance

In [50]:
var_list = []

layer = 1
for i in range (1, 7):
    print("Layer: ", layer)
    var = saved_model.network[layer].running_var
    print("Variance: \n", var)
    var_list.append(var.tolist())
    layer += 4

Layer:  1
Variance: 
 tensor([0.0275, 0.0068, 0.0039, 0.0110, 0.0023, 0.0042, 0.0256, 0.0363, 0.0131,
        0.0126, 0.0178, 0.0191, 0.0456, 0.0136, 0.0046, 0.0040, 0.0030, 0.0035,
        0.0093, 0.0120, 0.0206, 0.0053, 0.0087, 0.0059, 0.0110, 0.0012, 0.0105,
        0.0035, 0.0121, 0.0111, 0.0116, 0.0046])
Layer:  5
Variance: 
 tensor([1.7569, 1.9085, 1.7759, 3.0327, 2.2251, 1.6454, 2.1640, 2.4096, 4.1664,
        1.6708, 2.1257, 2.2535, 1.2966, 1.7898, 3.6126, 2.7613, 3.0436, 3.7944,
        1.1085, 2.9149, 2.5115, 3.0648, 1.7307, 3.6834, 2.8789, 1.6847, 2.7997,
        1.9957, 2.7438, 2.1079, 3.7478, 2.6339, 1.4438, 1.2327, 1.6549, 2.3948,
        4.3496, 5.5920, 1.5178, 2.9393, 1.9784, 3.2396, 4.1278, 3.0831, 2.0039,
        3.1332, 3.0581, 4.6341, 2.0594, 2.0667, 1.7251, 2.5642, 2.9523, 2.7562,
        2.3924, 3.0870, 3.1191, 2.9569, 2.6441, 1.9171, 3.6429, 3.3328, 3.3443,
        3.7973])
Layer:  9
Variance: 
 tensor([1.2772, 1.2991, 1.6149, 1.3832, 1.2255, 0.8917, 2.0233, 1.29

## Getting old gamma

In [51]:
old_gamma_list = []

layer = 1
for i in range (1, 7):
    print("Layer: ", layer)
    old_gamma = saved_model.network[layer].weight
    print("Gamma: \n", old_gamma)
    old_gamma_list.append(old_gamma.tolist())
    layer += 4

Layer:  1
Gamma: 
 Parameter containing:
tensor([0.8802, 0.9818, 1.1628, 0.8812, 1.0076, 0.8982, 0.7735, 0.8199, 0.8808,
        0.9961, 0.7846, 0.9566, 0.7756, 0.9585, 1.3335, 1.2013, 1.0810, 0.9633,
        1.0617, 1.1358, 0.8114, 1.0490, 1.2116, 1.2651, 0.9966, 1.0391, 1.0065,
        1.2038, 0.7744, 0.9251, 0.7799, 1.1656], requires_grad=True)
Layer:  5
Gamma: 
 Parameter containing:
tensor([0.9026, 0.9291, 0.8887, 1.2460, 0.9053, 0.8631, 0.8567, 0.9643, 0.8921,
        1.1546, 0.9224, 1.0618, 0.9288, 0.8785, 0.9215, 1.0495, 0.9683, 1.0919,
        0.8898, 1.0276, 0.9233, 0.8805, 0.9076, 0.8658, 0.9340, 0.8954, 1.0261,
        0.8431, 0.8797, 0.8677, 0.8439, 0.8788, 0.9026, 0.9383, 0.9463, 0.8351,
        1.0498, 0.8649, 0.8540, 1.0498, 0.9172, 1.1751, 1.1301, 0.9094, 0.9009,
        0.8684, 0.9251, 0.8552, 0.8573, 0.9091, 0.9171, 0.9142, 0.9923, 1.0395,
        0.9553, 1.0929, 1.0825, 1.1410, 1.2359, 1.2696, 0.9553, 0.9455, 0.9342,
        0.8883], requires_grad=True)
Layer:  9
Ga

## Getting old beta

In [52]:
old_beta_list = []

layer = 1
for i in range (1, 7):
    print("Layer: ", layer)
    old_beta = saved_model.network[layer].bias
    print("Beta: \n", old_beta)
    old_beta_list.append(old_beta.tolist())
    layer += 4

Layer:  1
Beta: 
 Parameter containing:
tensor([-0.1678,  0.1721,  0.0438, -0.1672,  0.0190,  0.1299, -0.2462, -0.0785,
         0.0004,  0.0707, -0.1068,  0.0842, -0.3400,  0.1122,  0.1407,  0.1833,
         0.0292,  0.0299,  0.0571,  0.0081, -0.1507,  0.1025,  0.0762,  0.1654,
         0.1398,  0.2156,  0.0607,  0.2683, -0.2458, -0.0476, -0.1522,  0.1774],
       requires_grad=True)
Layer:  5
Beta: 
 Parameter containing:
tensor([-0.1827, -0.2174, -0.2086, -0.2722, -0.1914, -0.2535, -0.2085, -0.1909,
        -0.2647, -0.3356, -0.2978, -0.3218, -0.1754, -0.1868, -0.0678, -0.1984,
        -0.2387, -0.2338, -0.2523, -0.2677, -0.2207, -0.0948, -0.1581, -0.1806,
        -0.1310, -0.1194, -0.2428, -0.2582, -0.1624, -0.2608, -0.2037, -0.2010,
        -0.2368, -0.2810, -0.2302, -0.2650, -0.2084, -0.2224, -0.2579, -0.2630,
        -0.2973, -0.2670, -0.2165, -0.1695, -0.2159, -0.2058, -0.2965, -0.2490,
        -0.2216, -0.1356, -0.3292, -0.2333, -0.2893, -0.2820, -0.2841, -0.3140,
        -0.2

## Getting modified parameters

### Formula to calculate new gamma and new beta

In [53]:
def new_gamma_formula(old_gamma, var):
    std_dev = (var+1e-05) ** 0.5
    new_gamma = old_gamma / std_dev
    return new_gamma

def new_beta_formula(old_gamma, mean, var, beta):
    std_dev = (var+1e-05) ** 0.5
    new_beta = -((old_gamma * mean) / (std_dev)) + beta
    return new_beta

### New gamma

In [54]:
new_gamma_list = []

for i in range(0,6):
    old_gamma_whole_list = old_gamma_list[i]
    var_whole_list = var_list[i]
    # print(old_gamma_whole_list)
    # print("\n",var_whole_list)
    # break
    temp = []
    if len(old_gamma_whole_list) != len(var_whole_list):
        print("Different length!")
    else:
        for j in range(0,len(old_gamma_whole_list)):
            old_gamma = old_gamma_whole_list[j]
            var = var_whole_list[j]
            # print("old_gamma: ", old_gamma)
            # print("var: ",var)
            # break
            new_gamma = new_gamma_formula(old_gamma, var)
            temp.append(new_gamma)
        new_gamma_list.append(temp)

### New beta

In [55]:
new_beta_list = []

for i in range(0,6):
    old_gamma_whole_list = old_gamma_list[i]
    var_whole_list = var_list[i]
    mean_whole_list = mean_list[i]
    old_beta_whole_list = old_beta_list[i]
    # print(old_gamma_whole_list)
    # print("\n",var_whole_list)
    # break
    temp = []
    if len(old_gamma_whole_list) != len(var_whole_list):
        print("Different length!")
    else:
        for j in range(0,len(old_gamma_whole_list)):
            old_gamma = old_gamma_whole_list[j]
            var = var_whole_list[j]
            mean = mean_whole_list[j]
            beta = old_beta_whole_list[j]
            # print("old_gamma: ", old_gamma)
            # print("var: ",var)
            # break
            new_beta = new_beta_formula(old_gamma, mean, var, beta)
            temp.append(new_beta)
        new_beta_list.append(temp)

## Function to save new gamma and new beta

In [56]:
def write_to_text_file(arr, path, file_name):
    # Converting to 1 dimensional array
    elements_of_new_arr = []
    for i in range (0, len(arr)):
        elements_of_new_arr.append(arr[i])
        
    # Converting to tensor
    torch_tensor = torch.tensor(elements_of_new_arr)
    
    # Writing to text file
    write_tensor_to_file(torch_tensor, path, file_name)

### Creating a directory to store new gamma 

In [57]:
new_gamma_path = create_if_not_exists(path_where_all_parameters_be_stored, "new_gamma")

In [58]:
new_gamma_path

'./PrunedResnetData/all_parameters/new_gamma/'

## Saving new gamma

In [59]:
layer = 1
for i in range(0,6):
    write_to_text_file(new_gamma_list[i], new_gamma_path, "P0-bn_layer_{}_new_gamma.txt".format(layer))
    layer += 1

### Creating a directory to store new beta

In [60]:
new_beta_path = create_if_not_exists(path_where_all_parameters_be_stored, "new_beta")

In [61]:
new_beta_path

'./PrunedResnetData/all_parameters/new_beta/'

## Saving new beta

In [62]:
layer = 1
for i in range(0,6):
    write_to_text_file(new_beta_list[i], new_beta_path, "P0-bn_layer_{}_new_beta.txt".format(layer))
    layer += 1

# Linear layers (fully connected layers)

### Getting weights of the linear layer

In [63]:
linear_weight_list = []

# 1st linear layer
linear_weight_list.append("################# LAYER #################")
linear_weight_list.append(saved_model.network[25].weight)

# 2nd linear layer
linear_weight_list.append("################# LAYER #################")
linear_weight_list.append(saved_model.network[28].weight)

### Getting bias of the linear layer

In [64]:
linear_bias_list = []

# 1st linear layer
linear_bias_list.append("################# LAYER #################")
linear_bias_list.append(saved_model.network[25].bias)

# 2nd linear layer
linear_bias_list.append("################# LAYER #################")
linear_bias_list.append(saved_model.network[28].bias)

### Saving weights to this path

In [65]:
weight_path

'./PrunedResnetData/all_parameters/weights/'

In [66]:
layer = 1
for i in range(0,len(linear_weight_list)):
    if type(linear_weight_list[i]) != str:
        linear_weight_tensor = linear_weight_list[i]
        write_tensor_to_file(linear_weight_tensor, weight_path, "P0-linear_layer_{}_weight.txt".format(layer))
        layer += 1

### Saving bias to this path

In [67]:
layer = 1
for i in range(0,len(linear_bias_list)):
    if type(linear_bias_list[i]) != str:
        linear_bias_tensor = linear_bias_list[i]
        write_tensor_to_file(linear_bias_tensor, bias_path, "P0-linear_layer_{}_bias.txt".format(layer))
        layer += 1